In [ ]:
# Import libraries and define global variables
import torch
from torch import nn, optim
from torch.nn.functional import cross_entropy, normalize
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from typing import Tuple, List

model_name = 'bert-base-uncased'
raw_data = pd.read_csv('drive/MyDrive/Colab Notebooks/CS412/data/train.csv')

In [ ]:
# Define classes
class DataToDataset(Dataset):
    def __init__(self, depPath:List[str], pair:List[Tuple[str, str]]):
        self.depPath = depPath
        self.pair = pair

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        return self.depPath[index], self.pair[index]

class PairEncoder(nn.Module):
    def __init__(self, config):
        super(PairEncoder, self).__init__()
        self.dropout = nn.Dropout(p=config.dropout)
        self.nonlinearity  = nn.ReLU()
        self.normalize = normalize if getattr(config, 'normalize_args', False) else (lambda x : x)
        self.arg_emb = nn.Embedding()
        
        self.pairEncoder = nn.Sequential(
            self.dropout, 
            nn.Linear(3 * config.d_args, config.d_args), 
            self.nonlinearity, 
            self.dropout, 
            nn.Linear(config.d_args, config.d_args), 
            self.nonlinearity, 
            self.dropout, 
            nn.Linear(config.d_args, config.d_args), 
            self.nonlinearity, 
            self.dropout, 
            nn.Linear(config.d_args, config.d_rels)
        )
        else:
            raise NotImplementedError()

    def forward(self, subjects:torch.Tensor, objects:torch.Tensor):
        subjects = self.normalize(subjects)
        objects = self.normalize(objects)
        return self.pairEncoder(torch.cat([subjects, objects, subjects * objects], dim=-1))

class PathEncoder(nn.Module):
    def __init__(self, config):
        super(PathEncoder, self).__init__()

